Create Ontology

In [1]:
from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
from  common.cleaner import clean_char

def _escape_value(text: str) -> str:
    """Escape the illegal characters for an ontology property"""
    if text is None:
        return None
    # function to escape XML character data
    text = escape(text)
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    text = text.replace('\f', '')
    text = text.replace('\b', '')
    text = text.replace('"', '')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace('{', '')
    text = text.replace('}', '')
    text = text.replace('#', '')
    text = text.replace('|', '')
    text = clean_char(text)
    return text

def _escape_iri(text: str) -> str:
    """For IRI, we replace space character by _"""
    if text is None:
        return None
    text = _escape_value(text)
    text = text.replace(' ', '_')
    text = text.replace('.', '_')
    return text


In [3]:
onto = get_ontology("http://test_fil_rouge.org/onto.owl")

In [4]:
# Create classes

with onto :
    class Author(Thing):
        pass
    class ConnectedAuthor(Author):
        pass

    class Article(Thing):
        # name of an article is the pdf_id in arxiv
        pass
    class InterestingAuthor(ConnectedAuthor):
        pass


In [5]:
# Data property

with onto :
    class has_number_citations(DataProperty):
        range = [int]

In [6]:
# Create new properties to create relations
with onto:
    class wrote_Article(Author >> Article):
        pass

    class quoted_Auteur(ObjectProperty):
        domain = [Author]
        range = [ConnectedAuthor, Author]

    class quoted_by(ConnectedAuthor >> Author):
        inverse_property = quoted_Auteur

    class wrote_by(Article >> Author):
        inverse_property = wrote_Article

    class quoted_in(ObjectProperty):
        domain = [ConnectedAuthor, Author]
        range = [Article]

        
    

Créer les instances

In [7]:
# Créer les auteurs et les articles d'abord
for i in range(20):

    Article("Article{}".format(i))


Bob = Author("Bob")
Bob.wrote_Article=[onto.Article.instances()[0]]
Arthur = Author("Arthur")
Arthur.wrote_Article=[onto.Article.instances()[0]]
David = Author("David")
David.wrote_Article=[onto.Article.instances()[1]]
Nassim = Author("Nassim")
Nassim.wrote_Article=[onto.Article.instances()[2]]
Safia = Author("Safia")
Safia.wrote_Article=[onto.Article.instances()[3]]



In [8]:
# References

Cécile = ConnectedAuthor('Cécile')
Cécile.quoted_in.append(Article.instances()[3])
Cécile.quoted_in.append(Article.instances()[0])
Nathalie= ConnectedAuthor('Nathalie')
for i in range(7):

    Nathalie.quoted_in.append(Article.instances()[i])
Julie =  ConnectedAuthor('Julie')
for i in range(3,6):

    Julie.quoted_in.append(Article.instances()[i])


In [9]:
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\Users\Maria\Documents\00-projets\fil-rouge-master\fil-rouge-master\.venv\lib\site-packages\owlready2\hermit;c:\Users\Maria\Documents\00-projets\fil-rouge-master\fil-rouge-master\.venv\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/Maria/AppData/Local/Temp/tmpld529nxk
* Owlready2 * HermiT took 0.9635756015777588 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [10]:
onto.save(file = "test_fil_rouge.owl")

Créer les règles

In [11]:
# Première règle : si un auteur a écrit un article et qu’un auteur cité est cité dans cet article, alors l’auteur cite l’auteur cité
with onto :
    rule = Imp()
    rule.set_as_rule("""wrote_Article(?r,?a), quoted_in(?c, ?r) -> quoted_Auteur(?a, ?c) """)

In [12]:
# Calcul nombre d'auteurs qui cites

for MonAuteur in Author.instances():
    nombre_de_citations = 0

    for citation in MonAuteur.quoted_in:

        nombre_de_citations +=1

    MonAuteur.has_number_citations=[nombre_de_citations]


In [13]:
# Calcul nombre d'auteurs qui cites

for Connected in ConnectedAuthor.instances():
    nombre_de_citations = 0

    for citation in Connected.quoted_in:

        nombre_de_citations +=1

    Connected.has_number_citations=[nombre_de_citations]


        
    

In [14]:
# un auteur qui a une relation quoted_in est un ConnectedAuthor

with onto :
    rule = Imp()
    rule.set_as_rule("""Author(?a) , quoted_in(?a, ?r) -> ConnectedAuthor(?a)""")

In [15]:
# Règle 2 : Connected Author qui est cité plus de 50 fois 
# (comptage de relations quoted_in pour ConnectedAuthor) 
# est un InterestingAuthor
with onto :
    rule = Imp()
    rule.set_as_rule("""ConnectedAuthor(?c) , has_number_citations(?c, ?x) , greaterThan(?x, 5) -> InterestingAuthor(?c)""")



In [16]:
onto.save(file = "test_fil_rouge.owl")

Réinitialisation de l'ontologie

In [33]:
# Case permet de réinitialiser les instances enregistrées
for individual in Author.instances(): 
    destroy_entity(individual)

for individual in ConnectedAuthor.instances(): 
    destroy_entity(individual)

for individual in Article.instances(): 
    destroy_entity(individual)